Checking Run 3 signal kinematics against Run 2

Author(s): Raghav Kansal

In [ ]:
import os
from collections import OrderedDict
import coffea
from coffea import nanoevents
import numpy as np
import awkward as ak
import pandas as pd
from hist.intervals import ratio_uncertainty

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# mplhep for CMS-style plots
import mplhep as hep

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 16})

In [ ]:
MAIN_DIR = "../../../"

plot_dir = f"{MAIN_DIR}/plots/SignalCheck/23Oct27"
_ = os.system(f"mkdir -p {plot_dir}")

In [ ]:
run2_sig_path = "/eos/uscms/store/user/lpcpfnano/cmantill/v2_3/2018/HH/GluGluToHHTo4B_node_cHHH1_TuneCP5_PSWeights_13TeV-powheg-pythia8/GluGluToHHTo4B_node_cHHH1_preUL/230217_205036/0000/nano_mc2018_1.root"

# Can't open this directly with Coffea for some reason...
# run3_sig_path = "root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/Run3Summer22EENanoAODv10/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/Poisson70KeepRAW_124X_mcRun3_2022_realistic_postEE_v1-v1/30000/d00363f4-0cac-410d-8fc7-bb6f60ccb6cd.root"

run3_sig_path = "/eos/uscms/store/user/rkansal/bbbb/nano/Run3Summer22EENanoAODv10/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/d00363f4-0cac-410d-8fc7-bb6f60ccb6cd.root"

In [ ]:
events_dict = OrderedDict()
events_dict["run2"] = nanoevents.NanoEventsFactory.from_root(
    run2_sig_path, schemaclass=nanoevents.NanoAODSchema
).events()
events_dict["run3"] = nanoevents.NanoEventsFactory.from_root(
    run3_sig_path, schemaclass=nanoevents.NanoAODSchema
).events()

In [ ]:
GEN_FLAGS = ["fromHardProcess", "isLastCopy"]
HIGGS_PDGID = 25

higgs_dict = {}
for key, events in events_dict.items():
    genvars = {}
    # finding the two gen higgs
    higgs = events.GenPart[
        (abs(events.GenPart.pdgId) == HIGGS_PDGID) * events.GenPart.hasFlags(GEN_FLAGS)
    ]
    higgs[ak.argsort(higgs.pt, axis=1, ascending=False)]

    higgs_children = higgs.children
    for i in range(2):
        bb = higgs_children[:, i]
        bb[ak.argsort(bb.pt, axis=1, ascending=False)]
        genvars[f"bb{i}"] = bb

    genvars["higgs"] = higgs
    higgs_dict[key] = genvars

Plot Gen Kinematics

In [ ]:
key_label_map = {
    "run2": "Run 2 HH4b",
    "run3": "Run 3 HH4b (TSG)",
}

higgs_var_map = {
    "pt": (np.linspace(0, 600, 31), r"$p_T$ (GeV)"),
    "eta": (np.linspace(-5, 5, 31), r"$\eta$"),
}

bb_var_map = {
    "pt": (np.linspace(0, 400, 21), r"$p_T$ (GeV)"),
}

In [ ]:
for var, (bins, var_label) in higgs_var_map.items():
    for i in range(2):
        fig, (ax, rax) = plt.subplots(
            2, 1, figsize=(12, 14), gridspec_kw=dict(height_ratios=[4, 1], hspace=0.07), sharex=True
        )

        hists = []

        for key, genvars in higgs_dict.items():
            h = np.histogram(genvars["higgs"][var][:, i], bins)
            hep.histplot(
                h,
                yerr=True,
                label=key_label_map[key],
                density=True,
                ax=ax,
            )

            hists.append(h[0])

        ax.set_ylabel("Events (A.U.)")
        ax.set_xlim(bins[0], bins[-1])
        ax.legend(fancybox=True)

        scale = np.sum(hists[0]) / np.sum(hists[1])
        yerr = ratio_uncertainty(hists[1], hists[0], "poisson-ratio") * scale
        hep.histplot(
            hists[1] / hists[0] * scale,
            bins,
            yerr=yerr,
            ax=rax,
            histtype="errorbar",
            color="black",
            capsize=4,
        )

        rax.hlines(1, -1000, bins[-1], color="lightgray", linestyles="--")
        rax.set_xlabel(f"Gen Higgs {i + 1} {var_label}")
        rax.set_ylabel("Run 3 / Run 2")
        rax.set_ylim([0, 2])
        # plt.legend()
        plt.savefig(f"{plot_dir}/higgs{i}_{var}.pdf", bbox_inches="tight")
        plt.show()

In [ ]:
for var, (bins, var_label) in bb_var_map.items():
    for i in range(2):
        for j in range(2):
            fig, (ax, rax) = plt.subplots(
                2,
                1,
                figsize=(12, 14),
                gridspec_kw=dict(height_ratios=[4, 1], hspace=0.07),
                sharex=True,
            )

            hists = []

            for key, genvars in higgs_dict.items():
                h = np.histogram(genvars[f"bb{i}"][var][:, j], bins)
                hep.histplot(
                    h,
                    yerr=True,
                    label=key_label_map[key],
                    density=True,
                    ax=ax,
                )
                hists.append(h[0])

            ax.set_ylabel("Events (A.U.)")
            ax.set_xlim(bins[0], bins[-1])
            ax.legend(fancybox=True)

            scale = np.sum(hists[0]) / np.sum(hists[1])
            yerr = ratio_uncertainty(hists[1], hists[0], "poisson-ratio") * scale
            hep.histplot(
                hists[1] / hists[0] * scale,
                bins,
                yerr=yerr,
                ax=rax,
                histtype="errorbar",
                color="black",
                capsize=4,
            )

            rax.hlines(1, -1000, bins[-1], color="lightgray", linestyles="--")
            rax.set_xlabel(f"Gen Higgs {i + 1} bb Daughter {j + 1} {var_label}")
            rax.set_ylabel("Run 3 / Run 2")
            rax.set_ylim([0, 2])

            plt.savefig(f"{plot_dir}/higgs{i}_bb{j}_{var}.pdf", bbox_inches="tight")
            plt.show()

Trigger comparison